# Code for Report

In [3]:
import pandas as pd
import plotly.plotly as py
from geopy.geocoders import Nominatim
import geopy.distance as gdist
import numpy as np
import collections
import seaborn as sns
import geopandas as gpd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import neighbors
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize

C:\Users\abhoo\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



In [4]:
# uses adapted version of function accessed online to get rid of NAN values an replace with -
# https://quant.stackexchange.com/questions/31094/removing-nan-values-in-python-quantopian
# Written by stack exchange user: André Christoffer Andersen 
# all other numeric float values have commas stripped
def clean_data(df, cols):
    for i in cols:
        df[i] = df[i].apply(lambda x: 
                            np.nan if x=='-' else float(x.replace(',', '')))
    return df

## Building the socio-economic dataset

In [5]:
wealth = pd.read_csv('socioeconomic.csv')
wealth = wealth[wealth['State']== 'VIC']
wealth['Suburb'] = wealth['Suburb'].apply(lambda x: x.upper())
# uses regex to get rid of parenthesis and its contents from suburb names
wealth['Suburb'] = wealth['Suburb'].str.replace(r"\(.*\)","")
clean_data(wealth,['Usual Resident Population'])
wealth = wealth.ix[:, ['Suburb', 'Usual Resident Population', 'Score', 'State']]

C:\Users\abhoo\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated



In [6]:
def filter_suburbs(df, filters):
    rem = []
    for val, row in df.iterrows():
        if (row['Suburb'] not in filters):
            rem.append(val)
    df.drop(rem, inplace = True)
    df.index = range(len(df.index))
    return df

In [7]:
def clean_up(df):
    #restricts dataset to contain only victorian data
    df = df[df['State']== 'VIC']
    #converts subrubs into uppercase
    df['Suburb'] = df['Suburb'].apply(lambda x: x.upper())
    #uses regex to get rid of parenthesis and its contents
    df['Suburb'] = df['Suburb'].str.replace(r"\(.*\)","")
    #gets rid of suburbs that are not in both datasets
    com_set = set(wealth['Suburb']).intersection(set(df['Suburb']))
    df = filter_suburbs(df, com_set)
    return df

In [8]:
resources = pd.read_csv('eco_resources.csv')
resources = clean_up(resources)
edu = pd.read_csv('education.csv')
edu = clean_up(edu)

C:\Users\abhoo\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\abhoo\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\abhoo\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [9]:
#integrates the minimum resources score and minimum education and income score
common_wealth = set(wealth['Suburb']).intersection(set(edu['Suburb']))
wealth = filter_suburbs(wealth, common)
edu = filter_suburbs(edu, common)
wealth.insert(loc = wealth.shape[1], column = 'min education and income score', value = edu['Min Score'])
wealth.insert(loc = wealth.shape[1], column = 'min resource score', value = resources['Min score'])

NameError: name 'common' is not defined

## Building the crime dataset

In [ ]:
crime = pd.read_csv('crime.csv', encoding  = 'ISO-8859-1')
suburbs = {}
discard = []
prev = ''
new_crime = pd.DataFrame({'Year': crime['Year ending September'],
                         'Suburb': crime['Suburb/Town Name'],
                         'Incidents': crime['Incidents Recorded']})
new_crime = new_crime[new_crime['Year'] == 2016]
new_crime = clean_data(new_crime, ['Incidents'])
new_crime = new_crime.groupby(['Year', 'Suburb']).sum()
new_crime.to_csv('newcrime1.csv', index = True)


In [ ]:
n_crime = pd.read_csv('newcrime1.csv')
common = set(wealth['Suburb']).intersection(set(n_crime['Suburb']))
#ensures both datasets only contain the same suburbs 
n_crime = filter_suburbs(n_crime, common)
wealth = filter_suburbs(wealth, common)

## Integrating datasets

In [ ]:
integrated = pd.DataFrame({'Suburb': n_crime['Suburb'],
                       'Socio-economic Ranking': wealth['Score'],
                       'Crime Incidents': n_crime['Incidents'],
                    'Population': wealth['Usual Resident Population'],
                          'Minimum Resource Score': wealth['min resource score'],
                          'Minimum education and income score': wealth['min education and income score']})

## Finding longitude and latitude of suburbs

In [ ]:
long_lat = {}
error_suburbs = []
rem = []
geolocator = Nominatim(timeout = None)
for y in integrated['suburb']:
    location = geolocator.geocode(y+ ' ' + "VIC")
    print (y)
    if location == None:
        error_suburbs.append(y)
        continue
    else:
        long_lat[y] = (location.longitude, location.latitude)
integrated = filter_suburbs(integrated, error_suburbs)

## Pearson Correlation and Mutual information

In [ ]:
#using the code from Workshop ipynb: Example of Pearson Correlation and mutual information.ipynb
def entropy(T):
    H=0
    N=sum(T)
    for i in range(len(T)):
       if T[i]>0: H+=-T[i]/N*np.log2(T[i]/N)
    return H
def mutualInfo(x,y):
    x=(np.asarray(x)).astype(int)
    y=(np.asarray(y)).astype(int)
    assert(len(x)==len(y))
    
    nx=max(x)
    Tx=np.zeros(nx+1)
    for i in range(len(x)):
        Tx[x[i]]+=1.0
    Hx=entropy(Tx)

    ny=max(y)
    Ty=np.zeros(ny+1)
    for i in range(len(y)):
        Ty[y[i]]+=1.0
    Hy=entropy(Ty)
    
    T=np.zeros((nx+1,ny+1))
    for i in range(len(x)):
        T[x[i],y[i]]+=1.0
    Hxy=0
    for i in range(nx+1):
        for j in range(ny+1):
            if T[i,j]>0:  Hxy+=-T[i,j]/len(x)*np.log2(T[i,j]/len(x))
    
    Hxgy=Hxy-Hy
    Hygx=Hxy-Hx
    minH=min(Hx,Hy)
    
    return {'Hx':Hx,'Hy':Hy,'Hx|y': Hxgy , 'Hy|x': Hygx ,'MI':Hx+Hy-Hxy,'NMI':(Hx+Hy-Hxy)/minH} 

print("Pearson r is ",integrated['Crime Incidents'].corr(integrated['Socio-economic Ranking']))
result=mutualInfo(integrated.loc[:,'Socio-economic Ranking'],integrated.loc[:,'Crime Incidents'])
print("Entropies and mutual information are", result)

## K-NN algorithm

In [ ]:
#creates a dataframe column called high crime where crime incidents above the 75th percentile
#range of data are considered high (represented by 1) and other lows (0)
high_crime = []
for val, row in integrated.iterrows():
    if row['Crime Incidents'] >= 150:#np.percentile(n['crime'], 75):
        high_crime.append(1)
    else:
        high_crime.append(0)
integrated.insert(loc = integrated.shape[1], column = 'high_crime', value = high_crime)

In [ ]:
#calculate accuracy score using K-NN algorithm from Workshop 8

##get just the features
data=integrated[[ 'Minimum education and income score', 'Minimum Resource Score', 'Population', 'Socio-economic Ranking']]


##get just the class labels
classlabel=integrated['high_crime']

##randomly select 66% of the instances to be training and the rest to be testing
# random state parameter ensures that everytime code is run, same split occurs
X_train, X_test, y_train, y_test = train_test_split(data,classlabel, train_size=0.66, random_state=42)

#normalise the data to have 0 mean and unit variance using the library functions.  This will help for later
#computation of distances between instances
scaler = preprocessing.StandardScaler()
scaler.fit(X_train) #each feature column between 0 and 1
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

print(X_train.shape)
print(X_test.shape)

knn = neighbors.KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred=knn.predict(X_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat.T, square = True, annot = True, fmt = 'd',
           cbar = False)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.savefig('falsepos.png')

## Outlier detection and analysis

In [ ]:
# outlier analysis done through adapting code from online
# https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-dataframe
# written by Stack Overflow user: user2708149 
def noise_outlier(df_col):
    median_value = df_col.median()
    smaller_v = df_col[df_col < median_value]
    larger_v = df_col[df_col > median_value]
    iqr = larger_v.median() - smaller_v.median()
    noise_up = larger_v.median() + (iqr*1.5)
    noise_low = smaller_v.median() - (iqr*1.5)
    max_upp  =  larger_v.median() + (iqr*3)
    max_low = smaller_v.median() - (iqr*3)
    noise = sum(((s < noise_low) & (s > max_low))) + sum(((s > noise_up) & (s < max_up)))
    outliers = sum( s > max_up) + sum(s < max_low)
    return noise, outliers

In [ ]:
# creates boxplot to show outliers
# varied the attributes as required
n['crime'].plot.box()
plt.title('Crime Outliers')
plt.savefig('crime outliers.png')
plt.show()

## Scatter plot with Linear regression line


In [ ]:
# created scatterplot with linear regression using
# accessed from https://github.com/OpenGenus/quark/blob/master/code/code/artificial_intelligence/src/Linear_Regression/linear_regression.py
# Written by github user: AdiChat
def estimate_coef(x, y):
    # number of observations/points
    n = np.size(x)

    # mean of x and y vector
    m_x, m_y = np.mean(x), np.mean(y)

    # calculating cross-deviation and deviation about x
    SS_xy = np.sum((x - m_x) * (y - m_y))
    SS_xx = np.sum(x*x - m_x*m_x)

    # calculating regression coefficients
    b_1 = SS_xy / SS_xx
    b_0 = m_y - b_1 * m_x

    return (b_0, b_1)

def plot_regression_line(xs, ys):
    # dev stands for deviation
    dev = estimate_coef(xs, ys)

    y_pred = []
    for x in xs:
        y_pred.append(dev[0] + dev[1] * x)

    # plotting the regression line
    plt.plot(xs, y_pred, color = "g")
new = integrated[integrated['crime']<400]
xarr = new['crime']
yarr = new['socio-economic status']

# Setting points as numpy arrays.
# It is more convenient this way for further process.
x = np.array(xarr)
y = np.array(yarr)

# Plotting points.
plt.scatter(x, y)

plot_regression_line(x, y)
plt.xlabel('Number of Crime Incidents')
plt.ylabel('Socio-economic ranking')
plt.tight_layout()
plt.savefig('scatter_regression_sm.png', dpi = 1080)
plt.show()

## Parallel Coordinates

In [ ]:
#plots the parallel coordinates of the suburbs
# can set the legend also to suburbs but too many suburbs so hard to read
# normalises the data
integrated['Crime Incidents'] = (integrated['Crime Incidents'] -integrated['Crime Incidents'].min())/(integrated['Crime Incidents'].max()-integrated['Crime Incidents'].min())
integrated['Socio-economic Ranking'] = (integrated['Socio-economic Ranking'] -integrated['Socio-economic Ranking'].min())/(integrated['Socio-economic Ranking'].max()-integrated['Socio-economic Ranking'].min())
integrated['Population'] = (integrated['Population'] -integrated['Population'].min())/(integrated['Population'].max()-integrated['Population'].min())
integrated['Minimum Resource Score'] = (integrated['Minimum Resource Score'] -integrated['Minimum Resource Score'].min())/(integrated['Minimum Resource Score'].max()-integrated['Minimum Resource Score'].min())
integrated['Minimum education and income score'] = (integrated['Minimum education and income score'] -integrated['Minimum education and income score'].min())/(integrated['Minimum education and income score'].max()-integrated['Minimum education and income score'].min())#the average for every day of the week is computed

n['name'] = ''

#plot the data using parallel coordinates
parallel_coordinates(n[['Crime Incidents','Socio-economic Ranking','Population','Minimum Resource Score','Minimum education and income score', 'name']],'name')
plt.show()

## Creating a geographic heatmap

In [ ]:
#converts a multiple dimension shapefile to 2D and makes sure that format
# is consistent with existing dataset (uppercase suburbs with no parenthesis)
unwanted = ["NSW", "ACT", "NT", "QLD", "SA", "WA"]
sub = gpd.read_file("./SSC_2011_AUST.shp")
sub['SSC_NAME'] = sub['SSC_NAME'].apply(lambda x: x.upper())

for i in unwanted:
    sub = sub[sub['SSC_NAME'].str.contains(i) == False]

commons = set(integrated['Suburb']).intersection(set(sub['SSC_NAME']))
sub = filter_suburbs(sub, commons)

v = shapefile.geometry.apply(lambda x: True if x != None else False)
shapefile = shapefile[v]
shapefile.to_file(output_filename, driver = 'ESRI Shapefile')

In [ ]:
#adapted from https://github.com/jamalmoir/notebook_playground/blob/master/uk_2015_houseprice.ipynb
# Github user name: jamalmoir
# building a geographic heatmap of victoria

#builds the basemap, which is Victoria in this case
fig, ax = plt.subplots(figsize=(100,100))
m = Basemap(projection='merc',llcrnrlat=-39.3866,urcrnrlat=-33.9813,\
            llcrnrlon=139.1832,urcrnrlon=150.9715,resolution='h')
m.drawmapboundary(fill_color='#46bcec')
m.fillcontinents(color='#f2f2f2',lake_color='#46bcec')
m.drawcoastlines()

# reads shapefile and merges with integrated dataset
m.readshapefile('out', 'area')
df_poly = pd.DataFrame({
        'shapes': [Polygon(np.array(shape), True) for shape in m.area],
        'area': [area['SSC_NAME'] for area in m.area_info]
    })
df_poly = df_poly.merge(integrated, on = 'area', how='inner')

# plots the heatmap
cmap = plt.get_cmap('PuRd')   
pc = PatchCollection(df_poly.shapes, zorder=2)
norm = Normalize()

pc.set_facecolor(cmap(norm(df_poly['crime'].fillna(0).values)))
ax.add_collection(pc)

mapper = matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)

mapper.set_array(df_poly['crime'])
plt.colorbar(mapper, shrink=0.4)
plt.title('Socio-economic ranking hotspots')
plt.savefig('crime_not_norm.png')
m

